## HNN TT No Standards v1

In [4]:
import torch
import torch.nn as  nn
import torch.nn.functional as F
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from enum import Enum

In [5]:
class CourseType(Enum):
    THOERY = "theory"
    THEORY_PRACTICAL = "theory_practical"

In [6]:
@dataclass
class Course:
    id: str
    name: str
    course_type: CourseType
    sessions_per_week: int
    session_duration: int
    practical_sessions: int = 0
    practical_duration: int = 0

In [7]:
@dataclass
class TimeSlot:
    day_order: int
    start_time: int
    duration: int

In [8]:
@dataclass
class TimeSlot:
    day_order: int
    start_time: int
    duration: int

In [9]:
@dataclass
class  Faculty:
    id: str
    name: str
    courses: List[str]
    availability: Dict[int, List[TimeSlot]]

In [10]:
class ScheduleState:
    def _init_(self, config):
        self.day_orders =  range(1, config['num_day_orders'] + 1)
        self.working_days  = config['working_days']
        self.time_slot = self._generate_time_slots(config)
        self.schedule_grid = {}
    
    def _generate_time_slots(self, config):
        slots = []
        day_start = config['day_start_minutes']
        day_end = config['day_end_minutes']
        slot_duration = config['slot_duration']
        while current_time + slot_duration <=  day_end:
            slots.append(current_time)
            current_time += slot_duration
        return slots

In [12]:
class ScheduleEmbedding (nn.Module):
    def __init__(self, config):
        super().__init__()
        self.day_order_embeddings = nn.Embedding(len(config['time_slots']), config['hidden_dim'])
        self.time_embedding = nn.Embedding(len(config['time_slots']), config['hidden_dim'])
        self.course_embedding = nn.Embedding(config['num_courses'], config['hidden_dim'])
        self.faculty_embedding = nn.Embedding(config['num_faculty'], config['hidden_dim'])

        self.fusion_network = nn.Sequential(
            nn.Linear(config['hidden_dim'] * 4, config['hidden_dim'] * 2),
            nn.ReLU(),
            nn.Linear(config['hidden_dim'] * 2, config['hidden_dim'])
        )

    def forward(self, schedule_state):
        day_embeds = self.day_order_embedding(schedule_state.day_orders)
        time_embeds = self.time_embedding(schedule_state.time_slots)
        course_embeds = self.course_embedding(schedule_state.courses)
        faculty_embeds = self.faculty_embedding(schedule_state.faculty)

        combined = torch.cat([day_embeds, time_embeds, course_embeds, faculty_embeds], dim = -1)
        return self.fusion_network(combined)


In [ ]:
class SchedulerNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embedding = ScheduleEmbedding(config)

        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model = config['hidden_dim'],
                nhead = config['num_heads'],
                dim_feedforward = config['hidden_dim'] * 4
            ),
            num_layers = config['num_layers']
        )

        self.schedule_head = nn.Sequential(
            nn.Linear(config['hidden_dim'], config['hidden_dim']),
            nn.ReLU(),
            nn.Linear(config['hidden_dim'], 1)
        )

    def forward(self, schedule_state):
        embedded = self.embedding(schedule_state)
        context = self.transformer(embedded)
        return self.schedule_head(context)
